# Fleet Analytics with Iceberg V3

This notebook is the **live demo companion** to the Iceberg V3 Comprehensive Guide. It contains all the hands-on exercises so you can follow along without switching between browser tabs.

## What You'll Explore

1. **Streaming Ingestion**: Real-time vehicle telemetry to VARIANT in Iceberg via Snowpipe Streaming
2. **Web API Ingestion**: Pull weather data from Open-Meteo API into VARIANT in Iceberg
3. **Batch JSON Ingestion**: Load maintenance logs with VARIANT to Iceberg
4. **Dynamic Tables**: Declarative transformation pipelines on Iceberg
5. **Security & Governance**: Sensitive data classification, Tagging, masking policies, data quality monitoring, and lineage on Iceberg tables
6. **Analytics**: Time-series and geospatial analytics with Iceberg V3 data types
7. **AI**: AI agents on top of Iceberg tables
8. **Business Continuity and Disaster Recovery**: Incremental cross-cloud and region replication with failover for Iceberg tables
9. **Interoperability**: Interoperable access from other engines with centralized fine-grained access controls for Apache Spark

## Prerequisites

Before running this notebook:
1. ✅ Run `setup.sh` to create all database objects
2. ✅ Start the streaming script: `python stream_telemetry.py`
3. ✅ Enable **OPEN_METEO_ACCESS** external integration (for API section)

In [ ]:
-- Set the context
USE ROLE ACCOUNTADMIN;
USE DATABASE FLEET_ANALYTICS_DB;
USE SCHEMA RAW;
USE WAREHOUSE FLEET_ANALYTICS_WH;

## Verify Setup

Before proceeding, let's verify all objects were created correctly by the setup script.

In [ ]:
-- Verify all Iceberg tables exist
SHOW ICEBERG TABLES IN DATABASE FLEET_ANALYTICS_DB;

-- Should show: VEHICLE_TELEMETRY_STREAM, MAINTENANCE_LOGS, SENSOR_READINGS, 
-- VEHICLE_LOCATIONS, VEHICLE_REGISTRY, API_WEATHER_DATA

In [ ]:
-- Verify Dynamic Iceberg Tables
SHOW DYNAMIC TABLES IN DATABASE FLEET_ANALYTICS_DB;

-- Should show: TELEMETRY_ENRICHED, DAILY_FLEET_SUMMARY

In [ ]:
-- Verify governance objects
SHOW MASKING POLICIES IN DATABASE FLEET_ANALYTICS_DB;
SHOW DATA METRIC FUNCTIONS IN DATABASE FLEET_ANALYTICS_DB;

-- Verify files are staged
LIST @RAW.LOGS_STAGE;

## Section 1: Streaming and Ingestion

The Python streaming script (`stream_telemetry.py`) simulates a fleet of 50 vehicles sending real-time telemetry events. Each event contains:

- **Location**: GPS coordinates with lat/lon
- **Speed**: Current vehicle speed in MPH
- **Engine metrics**: Temperature, oil pressure, fuel level
- **Diagnostics**: Warning indicators and error codes
- **Driver behavior**: Hard braking, acceleration events

The data is stored in the **VEHICLE_TELEMETRY_STREAM** Iceberg table using the **VARIANT** data type for flexible schema.

### Start the Streaming Script (Terminal)

If not already running, open a terminal and run:
```bash
cd iceberg-v3-tables-comprehensive-guide/assets
source iceberg_v3_demo_venv/bin/activate
python stream_telemetry.py
```

> **Connection Issues?** If you're on a VPN, you may need a network policy. Set `ENABLE_NETWORK_POLICY=true` in config.env and re-run setup.

In [ ]:
-- Check current count of streaming events
SELECT COUNT(*) AS event_count FROM VEHICLE_TELEMETRY_STREAM;

In [ ]:
-- View the latest streaming events with VARIANT extraction
SELECT 
    VEHICLE_ID,
    EVENT_TIMESTAMP,
    TELEMETRY_DATA:location:lat::FLOAT AS latitude,
    TELEMETRY_DATA:location:lon::FLOAT AS longitude,
    TELEMETRY_DATA:speed_mph::FLOAT AS speed_mph,
    TELEMETRY_DATA:engine:temperature_f::INT AS engine_temp,
    TELEMETRY_DATA:engine:fuel_level_pct::FLOAT AS fuel_level
FROM VEHICLE_TELEMETRY_STREAM
ORDER BY EVENT_TIMESTAMP DESC
LIMIT 10;

### Pull Data from Web APIs

Snowflake can connect directly to web APIs and ingest semi-structured responses into Iceberg tables with VARIANT.

**⚠️ Before running the Python cell below:**
1. Click the **Notebook settings** (gear icon, top right)
2. Select **External access**
3. Toggle ON **OPEN_METEO_ACCESS** integration
4. Click **Save**

This allows the notebook to access the Open-Meteo weather API.

In [ ]:
-- The table was created during setup
DESCRIBE TABLE API_WEATHER_DATA;

In [ ]:
-- Confirm it's empty
SELECT COUNT(*) AS current_count FROM API_WEATHER_DATA;

In [ ]:
# Fetch weather data from Open-Meteo API and load into Iceberg table
# NOTE: Ensure OPEN_METEO_ACCESS external integration is enabled in notebook settings
import requests
from datetime import datetime
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, parse_json, current_timestamp

session = get_active_session()

# Define cities for fleet operations
fleet_cities = [
    {"name": "Los Angeles", "lat": 34.0522, "lon": -118.2437},
    {"name": "Seattle", "lat": 47.6062, "lon": -122.3321},
    {"name": "Denver", "lat": 39.7392, "lon": -104.9903},
    {"name": "Chicago", "lat": 41.8781, "lon": -87.6298}
]

weather_records = []
for city in fleet_cities:
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": city["lat"],
        "longitude": city["lon"],
        "current": "temperature_2m,wind_speed_10m,precipitation",
        "hourly": "temperature_2m,precipitation_probability"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        weather_records.append({
            "CITY_NAME": city["name"],
            "LATITUDE": city["lat"],
            "LONGITUDE": city["lon"],
            "API_RESPONSE": response.text
        })
    else:
        print(f"Failed to fetch weather for {city['name']}: {response.status_code}")

# Create DataFrame and write to Iceberg table
if weather_records:
    df = session.create_dataframe(weather_records)
    df = df.with_column("WEATHER_DATA", parse_json(col("API_RESPONSE")))
    df.select(
        col("CITY_NAME"),
        col("LATITUDE"),
        col("LONGITUDE"),
        col("WEATHER_DATA"),
        current_timestamp().alias("INGESTED_AT")
    ).write.mode("append").save_as_table("API_WEATHER_DATA")
    print(f"Loaded weather data for {len(weather_records)} cities")
else:
    print("No weather data fetched. Check external access integration is enabled.")

In [ ]:
-- View the loaded weather data
SELECT 
    CITY_NAME,
    WEATHER_DATA:current:temperature_2m::FLOAT AS current_temp_c,
    WEATHER_DATA:current:wind_speed_10m::FLOAT AS wind_speed_kmh,
    WEATHER_DATA:current:precipitation::FLOAT AS precipitation_mm,
    INGESTED_AT
FROM API_WEATHER_DATA;

In [ ]:
-- Extract hourly forecast for a city
SELECT 
    CITY_NAME,
    f.value::FLOAT AS hourly_temp
FROM API_WEATHER_DATA,
LATERAL FLATTEN(input => WEATHER_DATA:hourly:temperature_2m) f
WHERE CITY_NAME = 'Seattle'
LIMIT 24;

### Batch Ingest JSON Logs

Fleet maintenance logs are collected as JSON files and loaded into Snowflake using **COPY INTO**. Each log file contains maintenance events with nested data:

```json
{
  "log_id": "LOG-001",
  "vehicle_id": "VH-1234",
  "event_type": "SCHEDULED_MAINTENANCE",
  "severity": "MEDIUM",
  "parts": [{"name": "oil_filter", "cost": 25.99}],
  "total_cost": 175.50
}
```

The entire JSON payload is stored in a **VARIANT** column (`LOG_DATA`) for schema flexibility.

#### Sample JSON Files

The setup script uploaded 10 sample maintenance log files to the internal stage `@LOGS_STAGE`.

In [ ]:
-- List the staged JSON files
LIST @LOGS_STAGE;

In [ ]:
-- Load JSON files into Iceberg table
COPY INTO MAINTENANCE_LOGS (LOG_ID, VEHICLE_ID, LOG_TIMESTAMP, LOG_DATA, SOURCE_FILE)
FROM (
    SELECT 
        $1:log_id::VARCHAR,
        $1:vehicle_id::VARCHAR,
        $1:log_timestamp::TIMESTAMP_NTZ,
        $1,
        METADATA$FILENAME
    FROM @LOGS_STAGE
)
FILE_FORMAT = (TYPE = 'JSON' STRIP_OUTER_ARRAY = TRUE)
PATTERN = '.*maintenance_log.*\.json';

In [ ]:
-- View the loaded maintenance logs with VARIANT data
SELECT 
    LOG_ID,
    VEHICLE_ID,
    LOG_TIMESTAMP,
    LOG_DATA:event_type::STRING AS event_type,
    LOG_DATA:severity::STRING AS severity,
    LOG_DATA:description::STRING AS description,
    LOG_DATA:total_cost::FLOAT AS total_cost
FROM MAINTENANCE_LOGS
ORDER BY LOG_TIMESTAMP DESC
LIMIT 10;

## Section 2: Declarative Transformation Pipelines

Dynamic Iceberg Tables provide **declarative, incremental transformations**. Instead of scheduling ETL jobs, you define the desired output and Snowflake automatically:

- ✅ Tracks source changes
- ✅ Incrementally refreshes data
- ✅ Maintains data freshness within your target lag

### Pipeline Architecture

```
VEHICLE_TELEMETRY_STREAM ─┬─→ TELEMETRY_ENRICHED ─→ DAILY_FLEET_SUMMARY
                          │         ↑
VEHICLE_REGISTRY ─────────┘         │
                                    │
SENSOR_READINGS ────────────────────┘
```

**TELEMETRY_ENRICHED**: Joins streaming telemetry with vehicle registry for context
**DAILY_FLEET_SUMMARY**: Aggregates to daily metrics by region and vehicle

In [ ]:
-- View all dynamic tables in the pipeline
SHOW DYNAMIC TABLES IN DATABASE FLEET_ANALYTICS_DB;

#### Explore the Pipeline in the UI

1. Navigate to **Data** → **Databases** → **FLEET_ANALYTICS_DB**
2. Filter by database: **FLEET_ANALYTICS_DB**
3. Select any Dynamic Table (e.g., `TELEMETRY_ENRICHED`)
4. Explore the tabs:
   - **Graph**: Visualize the full pipeline and dependencies
   - **Refresh History**: See incremental refresh operations
   - **Definition**: Review the declarative SQL

In [ ]:
-- Query enriched telemetry with vehicle details
SELECT 
    VEHICLE_ID,
    MAKE,
    MODEL,
    DRIVER_NAME,
    FLEET_REGION,
    speed_mph,
    engine_temp_f,
    engine_health_status,
    driving_behavior,
    EVENT_TIMESTAMP
FROM CURATED.TELEMETRY_ENRICHED
WHERE engine_health_status != 'NORMAL' OR driving_behavior = 'AGGRESSIVE'
ORDER BY EVENT_TIMESTAMP DESC
LIMIT 20;

## Section 3: Security, Governance, and BCDR

This section highlights Horizon Catalog as the central, unified, interoperable catalog with enterprise-grade security features. All governance capabilities work seamlessly with Iceberg V3 tables.

### PII Detection with Classification

Snowflake can automatically detect and classify sensitive data across your entire database, including Iceberg tables.

#### Classify the Schema

1. Navigate to **Data** → **Databases** → **FLEET_ANALYTICS_DB** → **RAW** (the schema)
2. Click the **three dots menu** (⋮) in the top right corner
3. Select **Classify and Tag Sensitive Data**
4. Select all Iceberg tables in the schema
5. Toggle **Automatically tag data** ON
6. Click **Classify and Tag Sensitive Data**

#### View Applied Tags

After classification completes:

1. Navigate to any table (e.g., `VEHICLE_REGISTRY`)
2. Click the **Columns** tab
3. Notice tags applied to sensitive columns like `DRIVER_NAME`, `DRIVER_EMAIL`, `DRIVER_PHONE`

Common tags that may be applied:
- `SEMANTIC_CATEGORY:NAME` - Personal names
- `SEMANTIC_CATEGORY:EMAIL` - Email addresses
- `SEMANTIC_CATEGORY:PHONE_NUMBER` - Phone numbers
- `SEMANTIC_CATEGORY:US_SSN` - Social Security Numbers

> **Tip**: You can also create [custom tags that automatically propagate](https://docs.snowflake.com/en/user-guide/object-tagging/work#define-a-tag-that-will-automatically-propagate) to existing and new downstream objects.

### Fine-Grained Access Control with Masking Policies

Masking policies protect sensitive data by dynamically transforming values based on the querying user's role. The **VEHICLE_REGISTRY** table contains driver PII:

| Column | Policy | Behavior |
|--------|--------|----------|
| DRIVER_NAME | PII_NAME_MASK | Admins see full name; analysts see "Jo****hn" |
| DRIVER_EMAIL | PII_EMAIL_MASK | Admins see full email; analysts see "jo****@****.com" |
| DRIVER_PHONE | PII_PHONE_MASK | Admins see full phone; analysts see "+1-555-***-**23" |

#### Apply Masking Policies (UI Method)

1. Navigate to **Data** → **Databases** → **FLEET_ANALYTICS_DB** → **RAW** → **VEHICLE_REGISTRY**
2. Click **Columns** tab
3. Click **+** next to a column's Policy field
4. Select the appropriate mask and click **Done**

Below we'll test masking behavior by switching roles.

In [ ]:
-- View all masking policies in the database
SHOW MASKING POLICIES IN DATABASE FLEET_ANALYTICS_DB;

In [ ]:
-- As ACCOUNTADMIN, see full PII values (unmasked)
SELECT VEHICLE_ID, DRIVER_NAME, DRIVER_EMAIL, DRIVER_PHONE
FROM VEHICLE_REGISTRY LIMIT 5;

In [ ]:
-- Create a test analyst role and grant it to the current user
CREATE ROLE IF NOT EXISTS FLEET_ANALYST;
GRANT USAGE ON DATABASE FLEET_ANALYTICS_DB TO ROLE FLEET_ANALYST;
GRANT USAGE ON SCHEMA FLEET_ANALYTICS_DB.RAW TO ROLE FLEET_ANALYST;
GRANT SELECT ON ALL TABLES IN SCHEMA FLEET_ANALYTICS_DB.RAW TO ROLE FLEET_ANALYST;
GRANT USAGE ON WAREHOUSE FLEET_ANALYTICS_WH TO ROLE FLEET_ANALYST;

-- Grant the role to the current user so we can switch to it
SET MY_USER = CURRENT_USER();
GRANT ROLE FLEET_ANALYST TO USER IDENTIFIER($MY_USER);

In [ ]:
-- Switch to analyst role and observe MASKED values
USE ROLE FLEET_ANALYST;
SELECT VEHICLE_ID, DRIVER_NAME, DRIVER_EMAIL, DRIVER_PHONE
FROM VEHICLE_REGISTRY LIMIT 5;

In [ ]:
-- Switch back to ACCOUNTADMIN for remaining operations
USE ROLE ACCOUNTADMIN;

### Data Quality Monitoring

Data Metric Functions (DMFs) validate data quality automatically. The Data Quality dashboard will initially be empty - DMFs run on a schedule and results populate after the first run.

> **Note**: To see results sooner, set `TRIGGER_ON_CHANGES` and insert data to trigger the DMF.

In [ ]:
-- Set DMFs to trigger when data changes
ALTER ICEBERG TABLE RAW.VEHICLE_LOCATIONS SET DATA_METRIC_SCHEDULE = 'TRIGGER_ON_CHANGES';
ALTER ICEBERG TABLE RAW.SENSOR_READINGS SET DATA_METRIC_SCHEDULE = 'TRIGGER_ON_CHANGES';

In [ ]:
-- Check which DMFs are attached to VEHICLE_LOCATIONS
SELECT * FROM TABLE(INFORMATION_SCHEMA.DATA_METRIC_FUNCTION_REFERENCES(
    REF_ENTITY_NAME => 'FLEET_ANALYTICS_DB.RAW.VEHICLE_LOCATIONS',
    REF_ENTITY_DOMAIN => 'TABLE'
));

In [ ]:
-- Insert a row to trigger the DMF (if using TRIGGER_ON_CHANGES)
INSERT INTO RAW.VEHICLE_LOCATIONS (LOCATION_ID, VEHICLE_ID, LOCATION_TIMESTAMP, LATITUDE, LONGITUDE, LOCATION_POINT, SPEED_MPH, HEADING_DEGREES, FLEET_REGION)
SELECT UUID_STRING(), VEHICLE_ID, CURRENT_TIMESTAMP(), LATITUDE, LONGITUDE, LOCATION_POINT, SPEED_MPH, HEADING_DEGREES, FLEET_REGION
FROM RAW.VEHICLE_LOCATIONS LIMIT 1;

In [ ]:
-- Check DMF results (may take a minute to populate after data insert)
SELECT 
    TABLE_NAME,
    METRIC_NAME,
    VALUE,
    MEASUREMENT_TIME
FROM SNOWFLAKE.LOCAL.DATA_QUALITY_MONITORING_RESULTS
WHERE TABLE_DATABASE = 'FLEET_ANALYTICS_DB'
ORDER BY MEASUREMENT_TIME DESC
LIMIT 20;

#### View Data Quality in the UI

After the DMF has run, view results in Snowsight:

1. Navigate to **Data** → **Databases** → **FLEET_ANALYTICS_DB** → **RAW** → **VEHICLE_LOCATIONS**
2. Click the **Data Quality** tab
3. View metrics by dimension (Freshness, Volume, Accuracy, etc.)

> **Tip**: If the dashboard is still empty, wait 1-2 minutes after inserting data and refresh the page.

### Data Lineage

Snowflake automatically tracks lineage for all objects, including Iceberg tables and Dynamic Tables.

#### View Lineage Graph

1. Navigate to **Data** → **Databases** → **FLEET_ANALYTICS_DB** → **RAW** → **MAINTENANCE_LOGS**
2. Click on the **Lineage** tab
3. Explore the lineage graph showing both upstream and downstream dependencies, including upstream lineage from an external telemetry messaging service

The lineage graph is valuable for:
- **Impact analysis**: Understanding what's affected when source data changes
- **Data quality investigation**: Tracing issues back to their source
- **Security auditing**: Understanding data flow for sensitive information

> 💡 **Try it**: Navigate to `VEHICLE_TELEMETRY_STREAM` and view its lineage to see how data flows through `TELEMETRY_ENRICHED` to `DAILY_FLEET_SUMMARY`.

### Business Continuity and Disaster Recovery

This section highlights Snowflake's enterprise-grade, out-of-the-box business continuity and disaster recovery capabilities that extend to Iceberg V3 tables.

> **Note**: Replication features require Business Critical Edition or higher.

#### Iceberg Table Replication Overview

Snowflake supports replicating Iceberg tables across regions and clouds, ensuring your data lakehouse is protected against regional outages. See [Iceberg Table Replication documentation](https://docs.snowflake.com/en/user-guide/tables-iceberg-replication) for details.

### Setting Up Replication via UI

Follow these steps to set up cross-region replication for your Iceberg tables:

##### Step 1: Create a Replication Group

1. Navigate to **Admin** → **Accounts**
2. Click on the **Replication** tab
3. Click **+ Replication Group** (or **+ Failover Group** for automatic failover)
4. Configure the group:
   - **Name**: `FLEET_ANALYTICS_REPLICATION`
   - **Replication Schedule**: Choose your RPO (e.g., every 10 minutes)
   - **Objects to replicate**: Select `FLEET_ANALYTICS_DB` database

##### Step 2: Select Target Account

1. Choose the target Snowflake account in a different region
2. If no secondary accounts exist, you'll need to create one first

##### Step 3: Configure Objects

1. Select all objects to replicate:
   - Databases (includes all Iceberg tables)
   - Warehouses (optional)
   - Roles and privileges (recommended for consistent access)

##### Step 4: Enable and Monitor

1. Click **Create** to enable replication
2. Monitor replication status in the Replication tab
3. View replication lag and sync history

#### Failover Testing

For Failover Groups (automatic failover):

1. In the Replication tab, select your failover group
2. Click **Failover** to initiate failover to secondary
3. Verify applications can connect to secondary account
4. Use **Failback** when primary is restored

## Section 4: Analytics and AI

This section demonstrates Snowflake's analytical capabilities on Iceberg V3 tables, including semi-structured data querying, time-series analysis, geospatial functions, and AI agents.

### Semi-Structured Data Analytics

Snowflake efficiently queries VARIANT data in Iceberg tables with automatic pruning and optimization.

In [ ]:
-- Extract nested fields from VARIANT with dot notation
SELECT 
    VEHICLE_ID,
    TELEMETRY_DATA:speed_mph::FLOAT AS speed,
    TELEMETRY_DATA:engine:rpm::INT AS engine_rpm,
    TELEMETRY_DATA:engine:temperature_f::INT AS engine_temp,
    TELEMETRY_DATA:diagnostics:check_engine::BOOLEAN AS check_engine_light,
    TELEMETRY_DATA:driver_behavior:hard_brake_count::INT AS hard_brakes
FROM RAW.VEHICLE_TELEMETRY_STREAM
WHERE TELEMETRY_DATA:speed_mph::FLOAT > 50
LIMIT 20;

In [ ]:
-- Aggregate on variant fields - find vehicles with check engine warnings
SELECT 
    TELEMETRY_DATA:diagnostics:check_engine::BOOLEAN AS check_engine,
    COUNT(*) AS event_count,
    ROUND(AVG(TELEMETRY_DATA:speed_mph::FLOAT), 1) AS avg_speed,
    COUNT(DISTINCT VEHICLE_ID) AS vehicle_count
FROM RAW.VEHICLE_TELEMETRY_STREAM
GROUP BY 1
ORDER BY event_count DESC;

In [ ]:
-- Run with profile to see pruning efficiency
SELECT 
    VEHICLE_ID,
    EVENT_TIMESTAMP,
    TELEMETRY_DATA:speed_mph::FLOAT AS speed
FROM VEHICLE_TELEMETRY_STREAM
WHERE VEHICLE_ID = 'VH-1234'
  AND TELEMETRY_DATA:engine:temperature_f::INT > 200;

-- Check query profile in History tab to see partition pruning statistics

### Time-Series Analytics and Forecasting

The `SENSOR_READINGS` table contains high-precision time-series data for analysis.

#### AS OF Join for Point-in-Time Analysis
Find the closest sensor reading **before** each maintenance event. This is invaluable for:
- Correlating maintenance with preceding anomalies
- Root cause analysis
- Predictive maintenance modeling

In [ ]:
-- AS OF join: Find closest sensor reading before each maintenance event
SELECT 
    m.VEHICLE_ID,
    m.LOG_TIMESTAMP AS maintenance_time,
    m.LOG_DATA:event_type::STRING AS maintenance_type,
    s.READING_TIMESTAMP AS closest_reading_time,
    s.ENGINE_TEMP_F,
    s.OIL_PRESSURE_PSI
FROM RAW.MAINTENANCE_LOGS m
ASOF JOIN RAW.SENSOR_READINGS s
    MATCH_CONDITION (m.LOG_TIMESTAMP >= s.READING_TIMESTAMP)
    ON m.VEHICLE_ID = s.VEHICLE_ID
WHERE m.LOG_DATA:severity::STRING IN ('CRITICAL', 'HIGH')
ORDER BY m.LOG_TIMESTAMP DESC
LIMIT 15;

#### Time-Series Aggregation with Windows

Window functions enable powerful time-series analysis directly in SQL:
- **Rolling averages** to smooth out noise
- **Running totals** for cumulative metrics
- **Lag/Lead** for period-over-period comparisons

In [ ]:
-- Calculate rolling averages for engine temperature
SELECT 
    VEHICLE_ID,
    READING_TIMESTAMP,
    ENGINE_TEMP_F,
    AVG(ENGINE_TEMP_F) OVER (
        PARTITION BY VEHICLE_ID 
        ORDER BY READING_TIMESTAMP 
        RANGE BETWEEN INTERVAL '1 HOUR' PRECEDING AND CURRENT ROW
    ) AS rolling_avg_temp,
    MAX(ENGINE_TEMP_F) OVER (
        PARTITION BY VEHICLE_ID 
        ORDER BY READING_TIMESTAMP 
        RANGE BETWEEN INTERVAL '1 HOUR' PRECEDING AND CURRENT ROW
    ) AS rolling_max_temp
FROM RAW.SENSOR_READINGS
ORDER BY VEHICLE_ID, READING_TIMESTAMP
LIMIT 50;

#### ML Forecasting for Predictive Maintenance

Snowflake's ML.FORECAST can build time-series forecast models directly on Iceberg tables. For Iceberg tables, use `SYSTEM$QUERY_REFERENCE` instead of `TABLE` reference.

> ⏱️ **Training Time**: Model training may take **2-5 minutes** depending on data volume and warehouse size. To speed up training:
> - Use a **larger warehouse** (e.g., MEDIUM or LARGE) - training scales with compute
> - Reduce the number of time series by filtering to specific vehicles
> - Limit historical data range (e.g., 14 days instead of 30)

In [ ]:
-- Create a forecast model for fuel consumption
-- For Iceberg tables, we must use SYSTEM$QUERY_REFERENCE instead of TABLE reference
CREATE OR REPLACE SNOWFLAKE.ML.FORECAST fuel_consumption_forecast(
    INPUT_DATA => SYSTEM$QUERY_REFERENCE('
        SELECT 
            VEHICLE_ID,
            READING_TIMESTAMP,
            FUEL_CONSUMPTION_GPH
        FROM RAW.SENSOR_READINGS
        WHERE READING_TIMESTAMP > DATEADD(''day'', -30, CURRENT_TIMESTAMP())
    '),
    TIMESTAMP_COLNAME => 'READING_TIMESTAMP',
    TARGET_COLNAME => 'FUEL_CONSUMPTION_GPH',
    SERIES_COLNAME => 'VEHICLE_ID'
);

In [ ]:
-- FORECAST: Generate a 7-day forecast (168 hours) and save to a table for visualization
CALL fuel_consumption_forecast!FORECAST(
    FORECASTING_PERIODS => 168,
    CONFIG_OBJECT => {'prediction_interval': 0.95}
);

-- Save forecast results to a table for visualization
CREATE OR REPLACE TEMPORARY TABLE FUEL_FORECAST_RESULTS AS
SELECT * FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));

-- Preview forecast results
SELECT * FROM FUEL_FORECAST_RESULTS ORDER BY SERIES, TS LIMIT 20;

##### Visualize Historical + Forecast Data

Use Streamlit in a Snowflake Notebook to plot historical data alongside the forecast:

In [ ]:
# 📈 Visualize Historical + Forecast Fuel Consumption
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Get historical data (last 30 days for one vehicle)
historical_df = session.sql("""
    SELECT 
        READING_TIMESTAMP AS ts,
        FUEL_CONSUMPTION_GPH AS value,
        'Historical' AS data_type
    FROM RAW.SENSOR_READINGS
    WHERE VEHICLE_ID = (SELECT SERIES FROM FUEL_FORECAST_RESULTS LIMIT 1)
      AND READING_TIMESTAMP > DATEADD('day', -30, CURRENT_TIMESTAMP())
    ORDER BY READING_TIMESTAMP
""").to_pandas()

# Get forecast data (column names vary - check with DESCRIBE first if needed)
forecast_df = session.sql("""
    SELECT 
        TS AS ts,
        FORECAST AS value,
        'Forecast' AS data_type
    FROM FUEL_FORECAST_RESULTS
    WHERE SERIES = (SELECT SERIES FROM FUEL_FORECAST_RESULTS LIMIT 1)
    ORDER BY TS
""").to_pandas()

# Get the vehicle ID for the title
vehicle_id = session.sql("SELECT SERIES FROM FUEL_FORECAST_RESULTS LIMIT 1").collect()[0][0]

st.subheader(f"📈 Fuel Consumption Forecast: {vehicle_id}")
st.caption("Historical data (30 days) + 7-day forecast")

# Normalize column names to lowercase
historical_df.columns = historical_df.columns.str.lower()
forecast_df.columns = forecast_df.columns.str.lower()

# Combine for plotting
combined_df = pd.concat([historical_df, forecast_df], ignore_index=True)
combined_df['ts'] = pd.to_datetime(combined_df['ts'])
combined_df = combined_df.sort_values('ts')

# Create the chart using Streamlit's native line chart
# Pivot data for multi-line chart
chart_data = combined_df.pivot_table(index='ts', columns='data_type', values='value', aggfunc='first')
st.line_chart(chart_data, use_container_width=True)

# Display metrics
col1, col2, col3 = st.columns(3)
col1.metric("Avg Historical (GPH)", f"{historical_df['value'].mean():.2f}")
col2.metric("Avg Forecast (GPH)", f"{forecast_df['value'].mean():.2f}")
change = ((forecast_df['value'].mean() - historical_df['value'].mean()) / historical_df['value'].mean()) * 100
col3.metric("Projected Change", f"{change:+.1f}%")

st.caption("💡 The forecast helps identify vehicles likely to have higher fuel consumption, enabling proactive maintenance scheduling.")

### Geospatial Analytics with H3 Indexing

H3 is Uber's hierarchical geospatial indexing system. Snowflake's H3 functions enable:
- Aggregating vehicles by geographic cell
- Efficient spatial joins and clustering
- Multi-resolution analysis (cells at different zoom levels)

In [ ]:
-- Calculate distance between consecutive positions
-- Note: LAG doesn't support GEOGRAPHY directly, so we use lat/lon coordinates
SELECT 
    VEHICLE_ID,
    LOCATION_TIMESTAMP,
    LATITUDE,
    LONGITUDE,
    LAG(LATITUDE) OVER (PARTITION BY VEHICLE_ID ORDER BY LOCATION_TIMESTAMP) AS prev_lat,
    LAG(LONGITUDE) OVER (PARTITION BY VEHICLE_ID ORDER BY LOCATION_TIMESTAMP) AS prev_lon,
    ST_DISTANCE(
        LOCATION_POINT,
        ST_MAKEPOINT(
            LAG(LONGITUDE) OVER (PARTITION BY VEHICLE_ID ORDER BY LOCATION_TIMESTAMP),
            LAG(LATITUDE) OVER (PARTITION BY VEHICLE_ID ORDER BY LOCATION_TIMESTAMP)
        )
    ) / 1609.34 AS distance_miles  -- Convert meters to miles
FROM VEHICLE_LOCATIONS
ORDER BY VEHICLE_ID, LOCATION_TIMESTAMP
LIMIT 50;

In [ ]:
-- Find vehicles inside a geofence (Los Angeles area)
WITH geofence AS (
    SELECT ST_MAKEPOLYGON(TO_GEOGRAPHY(
        'LINESTRING(-118.7 33.5, -118.7 34.5, -117.7 34.5, -117.7 33.5, -118.7 33.5)'
    )) AS la_area
)
SELECT 
    v.VEHICLE_ID,
    v.LOCATION_TIMESTAMP,
    ST_X(v.LOCATION_POINT) AS longitude,
    ST_Y(v.LOCATION_POINT) AS latitude,
    v.FLEET_REGION
FROM VEHICLE_LOCATIONS v, geofence g
WHERE ST_WITHIN(v.LOCATION_POINT, g.la_area)
ORDER BY v.LOCATION_TIMESTAMP DESC
LIMIT 20;

In [ ]:
-- Geospatial: Aggregate vehicles by H3 cell (resolution 6)
SELECT 
    H3_POINT_TO_CELL_STRING(LOCATION_POINT, 6) AS h3_cell,
    FLEET_REGION,
    COUNT(DISTINCT VEHICLE_ID) AS vehicle_count,
    ROUND(AVG(SPEED_MPH), 1) AS avg_speed
FROM RAW.VEHICLE_LOCATIONS
WHERE LOCATION_TIMESTAMP > DATEADD('hour', -24, CURRENT_TIMESTAMP())
GROUP BY 1, 2
ORDER BY vehicle_count DESC
LIMIT 20;

In [ ]:
# Visualize fleet locations on an interactive map using Streamlit
import streamlit as st
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Query location data with H3 cell centers (use ST_X/ST_Y for GEOGRAPHY type)
df = session.sql("""
    SELECT 
        H3_POINT_TO_CELL_STRING(LOCATION_POINT, 6) AS h3_cell,
        ST_X(H3_CELL_TO_POINT(H3_POINT_TO_CELL_STRING(LOCATION_POINT, 6))) AS longitude,
        ST_Y(H3_CELL_TO_POINT(H3_POINT_TO_CELL_STRING(LOCATION_POINT, 6))) AS latitude,
        FLEET_REGION,
        COUNT(DISTINCT VEHICLE_ID) AS vehicle_count,
        ROUND(AVG(SPEED_MPH), 1) AS avg_speed
    FROM RAW.VEHICLE_LOCATIONS
    WHERE LOCATION_TIMESTAMP > DATEADD('hour', -24, CURRENT_TIMESTAMP())
    GROUP BY 1, 2, 3, 4
    ORDER BY vehicle_count DESC
    LIMIT 100
""").to_pandas()

st.subheader("🗺️ Fleet Vehicle Distribution")
st.caption("Each point represents an H3 cell with vehicle activity in the last 24 hours")

# Display metrics
col1, col2, col3 = st.columns(3)
col1.metric("Total H3 Cells", len(df))
col2.metric("Total Vehicles", int(df['VEHICLE_COUNT'].sum()))
col3.metric("Avg Speed", f"{df['AVG_SPEED'].mean():.1f} mph")

# Interactive map visualization - point size scales with vehicle count
st.map(df, latitude='LATITUDE', longitude='LONGITUDE', size='VEHICLE_COUNT')

# Show data table
st.dataframe(df[['FLEET_REGION', 'VEHICLE_COUNT', 'AVG_SPEED']].head(10))

### AI Agents with Snowflake Intelligence

The setup script creates a Cortex Agent (`FLEET_ANALYTICS_AGENT`) that enables natural language querying of your fleet data.

> **Note**: Cortex Agents require Enterprise Edition or higher.

#### Region Availability and Cross-Region Inference

Cortex Agents and Cortex Analyst rely on LLMs that may not be available in all Snowflake regions. If you see an error like:

```
None of the preferred models are authorized or available in your region...
```

You have two options:

1. **Enable Cross-Region Inference** (recommended): This allows Snowflake to route AI requests to regions where models are available, while keeping your data secure.

In [ ]:
-- Enable cross-region inference for your account
USE ROLE ACCOUNTADMIN;
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'ANY_REGION';

2. **Use a supported region**: Deploy your Snowflake account in a region with native LLM support.

For more details, see:
- [Cortex Analyst Region Availability](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst#region-availability)
- [Cross-Region Inference](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cross-region-inference)

> **Privacy Note**: With cross-region inference, only the AI inference is routed cross-region. Your underlying data remains in your account's region and is protected by Snowflake's governance controls.

#### Using the Fleet Analytics Agent

1. Navigate to **Cortex AI** → **Agents** in the left sidebar
2. Select **FLEET_ANALYTICS_AGENT**
3. Start asking questions in natural language:

**Example queries:**

- "Which vehicles had the highest fuel consumption last week?"
- "Show me all critical maintenance events"
- "What's the average speed by fleet region?"
- "Find vehicles with check engine warnings"
- "Which drivers have the most hard braking events?"
- "Show vehicle health scores below 60"
- "How many vehicles are in each H3 cell in California?"

The agent understands your Iceberg table schema including:
- **VARIANT columns**: Extracts nested JSON fields using colon notation
- **GEOGRAPHY columns**: Uses H3 and ST_* functions for geospatial analysis
- **Time-series data**: Performs ASOF joins and window functions

#### Agent Configuration

The agent was created with access to these tables:

| Table | Description |
|-------|-------------|
| `RAW.VEHICLE_TELEMETRY_STREAM` | Real-time streaming telemetry (VARIANT) |
| `RAW.VEHICLE_LOCATIONS` | Geospatial positions (GEOGRAPHY) |
| `RAW.SENSOR_READINGS` | High-precision time-series sensor data |
| `RAW.MAINTENANCE_LOGS` | Maintenance events (VARIANT) |
| `RAW.VEHICLE_REGISTRY` | Vehicle and driver master data |
| `ANALYTICS.DAILY_FLEET_SUMMARY` | Daily aggregated metrics |
| `ANALYTICS.VEHICLE_HEALTH_SCORE` | Calculated health scores |

To view or modify the agent configuration:

## Section 5: Interoperability

This section demonstrates that external engines like Apache Spark can access Iceberg V3 tables managed by Snowflake. Horizon provides temporary, scoped storage credentials and enforces centralized row/column-level access controls for any engine.

### Prerequisites for Spark

Ensure you have:
- Conda installed
- The repository cloned (`sfguide-iceberg-v3-comprehensive`)

### Start the Spark Environment

The setup script created a Conda environment with Spark 4.0+ and all necessary dependencies:

```bash
cd iceberg-v3-tables-comprehensive-guide/assets

# Activate the Spark environment
conda activate fleet-spark

# Start Jupyter notebook
jupyter notebook notebooks/spark_iceberg_interop.ipynb
```

With `spark_iceberg_interop.ipynb` open in your browser, follow the instructions in the notebook.

## Cleanup
Run the cell below to remove all objects created by this guide.

In [ ]:
USE ROLE ACCOUNTADMIN;

-- Drop the database (this removes all tables, schemas, and objects)
DROP DATABASE IF EXISTS FLEET_ANALYTICS_DB;

-- Drop the external volume (if you created one)
DROP EXTERNAL VOLUME IF EXISTS FLEET_ICEBERG_VOL;

-- Drop the warehouse
DROP WAREHOUSE IF EXISTS FLEET_ANALYTICS_WH;

-- Drop roles
DROP ROLE IF EXISTS FLEET_ANALYST;
DROP ROLE IF EXISTS FLEET_ENGINEER;
DROP ROLE IF EXISTS FLEET_ADMIN;

-- If you created replication groups, remove them
DROP REPLICATION GROUP IF EXISTS FLEET_ANALYTICS_REPLICATION;

-- Remove network policy from user (if applied)
ALTER USER <your_username> UNSET NETWORK_POLICY;
DROP NETWORK POLICY IF EXISTS FLEET_STREAMING_POLICY;

-- External access integration
DROP INTEGRATION IF EXISTS OPEN_METEO_ACCESS;

SELECT 'Uncomment the commands above to cleanup' AS NOTE;